In [8]:
import json
import galois

import sha3

def numbers_to_hash(numbers, field) -> int:
    """Hash a number."""
    engine = sha3.keccak_256()
    for number in numbers:
        if isinstance(number, tuple):
            x, y, z = number
            engine.update(bytes(hex(int(x)), 'utf-8'))
            engine.update(bytes(hex(int(y)), 'utf-8'))
            engine.update(bytes(hex(int(z)), 'utf-8'))
        else:
            engine.update(bytes(hex(int(number)), 'utf-8'))
    return field(int(engine.hexdigest(), 16) % field.order)

encrypted = False

proof = json.load(open("proof.json", "r"))
p = proof.pop("Fp")

Fp = galois.GF(p)

proof = {k: Fp(v) for k, v in proof.items()}
print(proof)

circuit = json.load(open("circuit.json", "r"))
circuit.pop("Fp")
for k, v in circuit.items():
    if k in ["tau", "k1", "k2", "Fp", "omega", "n"]:
        circuit[k] = Fp(v)
    else:
        circuit[k] = galois.Poly(coeffs=v, field=Fp)

QM = circuit["QM"]
QL = circuit["QL"]
QR = circuit["QR"]
QO = circuit["QO"]
QC = circuit["QC"]
S1 = circuit["S1"]
S2 = circuit["S2"]
S3 = circuit["S3"]
Zh = circuit["Zh"]
L1 = circuit["L1"]
PI = circuit["PI"]
tau = circuit["tau"]
k1 = circuit["k1"]
k2 = circuit["k2"]
n = int(circuit["n"])
omega = circuit["omega"]

round1 = [proof["A"], proof["B"], proof["C"]]
round2 = [proof["Z"]]
round3 = [proof["Tl"], proof["Tm"], proof["Th"]]
round4 = [proof["a_zeta"], proof["b_zeta"], proof["c_zeta"], proof["s1_zeta"], proof["s2_zeta"], proof["z_omega_zeta"]]
round5 = [proof["Wzeta"], proof["Womega_zeta"]]

{'A': GF(219, order=241), 'B': GF(14, order=241), 'C': GF(167, order=241), 'Z': GF(72, order=241), 'Tl': GF(94, order=241), 'Tm': GF(36, order=241), 'Th': GF(156, order=241), 'Wzeta': GF(225, order=241), 'Womega_zeta': GF(34, order=241), 'a_zeta': GF(147, order=241), 'b_zeta': GF(239, order=241), 'c_zeta': GF(196, order=241), 's1_zeta': GF(160, order=241), 's2_zeta': GF(46, order=241), 'z_omega_zeta': GF(221, order=241)}


# Verifier

In [9]:
# These evaluations are calculated beforehand during the setup phase
qm_exp = QM(tau)
ql_exp = QL(tau)
qr_exp = QR(tau)
qo_exp = QO(tau)
qc_exp = QC(tau)
s1_exp = S1(tau)
s2_exp = S2(tau)
s3_exp = S3(tau)

# Values provided by the prover (round 1 to 5) is a proof.
a_exp = round1[0]
b_exp = round1[1]
c_exp = round1[2]

z_exp = round2[0]

tl_exp = round3[0]
tm_exp = round3[1]
th_exp = round3[2]

# Note: verifier has to verify that the following values are in the correct Fp field
a_zeta, b_zeta, c_zeta, s1_zeta, s2_zeta, z_omega_zeta = round4
a_zeta = a_zeta + Fp(1)

w_zeta_exp = round5[0]
w_omega_zeta_exp = round5[1]

# Note: verifier has to verify that the following values are on the curve
if encrypted:
    validate_point(qm_exp)
    validate_point(ql_exp)
    validate_point(qr_exp)
    validate_point(qo_exp)
    validate_point(qc_exp)
    validate_point(z_exp)
    validate_point(s1_exp)
    validate_point(s2_exp)
    validate_point(s3_exp)
    validate_point(tl_exp)
    validate_point(tm_exp)
    validate_point(th_exp)
    validate_point(a_exp)
    validate_point(b_exp)
    validate_point(c_exp)
    validate_point(w_zeta_exp)
    validate_point(w_omega_zeta_exp)

beta = numbers_to_hash(round1 + [0], Fp)
gamma = numbers_to_hash(round1 + [1], Fp)
alpha = numbers_to_hash(round1 + round2, Fp)
zeta = numbers_to_hash(round1 + round2 + round3, Fp)
v = numbers_to_hash(round1 + round2 + round3 + round4, Fp)
u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, Fp)

In [10]:
Zh_z = Zh(zeta)
L1_z = L1(zeta)
PI_z = PI(zeta)

r0 = (PI_z - L1_z * alpha**2 -
    (a_zeta + beta * s1_zeta + gamma) *
    (b_zeta + beta * s2_zeta + gamma) *
    (c_zeta + gamma) * z_omega_zeta * alpha)

In [11]:
D_exp = (qm_exp * a_zeta * b_zeta +
        ql_exp * a_zeta +
        qr_exp * b_zeta +
        qo_exp * c_zeta +
        qc_exp)

D_exp += (z_exp * (
        (a_zeta + beta * zeta + gamma) *
        (b_zeta + beta * zeta * k1 + gamma) *
        (c_zeta + beta * zeta * k2 + gamma) * alpha
        + L1_z * alpha**2 + u))

D_exp -= (s3_exp *
        (a_zeta + beta * s1_zeta + gamma) *
        (b_zeta + beta * s2_zeta + gamma) * 
        alpha * beta * z_omega_zeta)

D_exp -= ((tl_exp + 
        tm_exp * zeta**n  +
        th_exp * zeta**(2*n)) *
        Zh_z)

In [12]:
F_exp = (D_exp + 
        a_exp * v +
        b_exp * v**2 +
        c_exp * v**3 +
        s1_exp * v**4 +
        s2_exp * v**5)

print(f"F_exp = {F_exp}")

F_exp = 43


In [13]:
E_exp = (-r0 +
        v * a_zeta +
        v**2 * b_zeta +
        v**3 * c_zeta +
        v**4 * s1_zeta +
        v**5 * s2_zeta +
        u * z_omega_zeta)

if encrypted:
        E_exp = G1 * E_exp

print(f"E_exp = {E_exp}")

E_exp = 132


In [14]:
e1 = w_zeta_exp + w_omega_zeta_exp * u
e2 = (w_zeta_exp * zeta + w_omega_zeta_exp * (u * zeta * omega) +
    F_exp + (E_exp * Fp(p-1)))

if encrypted:
    pairing1 = tau.tau2.pair(e1)
    pairing2 = G2.pair(e2)

    print(f"pairing1 = {pairing1}")
    print(f"pairing2 = {pairing2}")

    assert pairing1 == pairing2, f"pairing1 != pairing2"
else:
    print("\n\n--- e1, e2 ---")
    print(f"e1 = {e1 * tau} = {e1} * tau")
    print(f"e2 = {e2}")
    assert e1 * tau == e2



--- e1, e2 ---
e1 = 47 = 209 * tau
e2 = 131


AssertionError: 

In [ ]:
from sympy import symbols, Eq, solve

# Define the symbols
X, Y = symbols('X Y')

# Constants are assumed to be known and are represented as symbols for generality
A, B, C, _u, z, w = symbols('A B C u z w')

# Define the equations based on the provided system
eq1 = Eq(X + _u*Y, A)
eq2 = Eq(z * X + _u*z*w*Y + C, B)

# Solve the system of equations
solutions = solve((eq1, eq2), (X, Y))
solutions

_A = e1
_B = e2
_C = F_exp + (E_exp * Fp(p-1))
# u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, Fp)
# u = Fp.Random()

y = (_B - _C - _A * zeta) / (u * omega * zeta - u  *zeta)

print(f"y = {y}")
print(f"w_omega_zeta_exp = {w_omega_zeta_exp}")

x = (_A* omega *zeta - _B + _C) / (omega * zeta - zeta)

print(f"x = {x}")
print(f"w_zeta_exp = {w_zeta_exp}")

assert x == w_zeta_exp

e1 = x + y * u
e2 = (x * zeta + y * (u * zeta * omega) +
    F_exp + (E_exp * Fp(p-1)))

assert e1 * tau == e2